In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
proxies = {
    "http": "http://127.0.0.1:2080",  # Replace with your proxy address
    "https": "http://127.0.0.1:2080"  # For HTTPS requests as well
}


In [3]:
res = requests.get("https://core.telegram.org/bots/api", proxies=proxies)

In [7]:
pattern = r'<h4><a class=\"anchor\" name=\".*?\" href=\".*?\"><i class=\"anchor-icon\"><\/i><\/a>(.*?)<\/h4>\s+<p>(.*?)<\/p>\s+<table class=\"table\">\s+<thead>\s+<tr>\s+<th>Parameter<\/th>\s+<th>Type<\/th>\s+<th>Required<\/th>\s+<th>Description<\/th>\s+<\/tr>\s+<\/thead>\s+<tbody.*?>([\s\S]*?)<\/tbody>'

In [8]:
all_methods = re.findall(pattern, res.text)

In [9]:
table_body_pattern = r'<tr>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+</tr>'

In [10]:
max_char_pattern = r'-(\d+) characters'

In [11]:
def text_cleaner(text):
    return BeautifulSoup(text, 'html.parser').get_text().replace('"', '\\"')

In [24]:
with open('../component/telegram/models.py', 'w') as file:
    file.write("""from django.contrib.postgres.fields import ArrayField
from typing import List

from django.conf import settings
from django.contrib.contenttypes.fields import GenericForeignKey
from django.contrib.contenttypes.models import ContentType
from django.contrib.postgres.fields import ArrayField
from django.db import models
from django.db.models import Q
from django.forms.models import model_to_dict
        
class Component(models.Model):
    class ComponentType(models.TextChoices):
        TELEGRAM = "TELEGRAM", "Telegram API Component"
        TRIGGER = "TRIGGER", "Trigger Component"
        CONDITIONAL = "CONDITIONAL", "Conditional Component"
        CODE = "CODE", "Code Component"
        STATE = "STATE", "State Component"

    component_type = models.CharField(
        max_length=20,
        choices=ComponentType.choices,
        default=ComponentType.TELEGRAM,
        help_text="Type of the component",
    )

    def save(self, *args: list, **kwargs: dict) -> None:
        if self.pk is None:
            self.component_content_type = ContentType.objects.get(
                model=self.__class__.__name__.lower(),
            )
        super().save(*args, **kwargs)

    component_content_type = models.ForeignKey(
        ContentType,
        on_delete=models.CASCADE,
        null=True,
        blank=True,
    )
    component_name = models.CharField(
        max_length=255,
        null=True,
    )  # in order to not interfere with some component 'name' field, I added redundant 'component'

    bot = models.ForeignKey("bot.Bot", on_delete=models.CASCADE)

    previous_component = models.ForeignKey(
        "Component",
        on_delete=models.SET_NULL,
        null=True,
        blank=True,
        related_name="next_component",
    )

    position_x = models.FloatField(null=False, blank=False)
    position_y = models.FloatField(null=False, blank=False)

    def __str__(self) -> str:
        return self.component_name or "Empty Component"

    class Meta:
        pass

    @property
    def code_function_name(self) -> str:  # -> name of the function in generated code
        return f"{self.__class__.__name__.lower()}_{self.pk}"

    def _get_file_params(self, underlying_object) -> str:
        file_params = ""
        for field in underlying_object._meta.get_fields():
            if isinstance(field, models.FileField):
                file_instance = getattr(underlying_object, field.name)
                if file_instance and hasattr(file_instance, "url"):
                    full_url = f"{settings.SITE_URL}{file_instance.url}"
                    file_params = f"{field.name}='{full_url}'"
                    setattr(underlying_object, field.name, None)
        return file_params

    def _get_method_name(self, class_name: str) -> str:
        method = ""
        for c in class_name:
            if c.isupper():
                method += "_"
            method += c.lower()
        return method.lstrip("_")

    def _generate_keyboard_code(self, keyboard) -> list[str]:
        if not isinstance(keyboard, InlineKeyboardMarkup):
            return []

        code = ["    builder = InlineKeyboardBuilder()"]
        for k in keyboard.inline_keyboard.all():
            code.append(
                f"    builder.button(text='{k.text}', callback_data='{k.callback_data}')",
            )
        code.append("    keyboard = builder.as_markup()")
        return code

    def _format_code_component(self, underlying_object) -> list[str]:
        try:
            import black

            formatted_code = black.format_str(underlying_object.code, mode=black.Mode())
            return [f"    {formatted_code}"]
        except Exception as e:
            return [
                f"    # Original code failed black formatting: {str(e)}",
                f"    # {underlying_object.code}",
                "    pass",
            ]

    def _get_component_params(
        self,
        underlying_object,
        keyboard,
        file_params: str,
    ) -> str:
        excluded_fields = {
            "id",
            "component_ptr",
            "component_ptr_id",
            "timestamp",
            "object_id",
            "component_type",
            "content_type",
            "component_content_type",
            "bot",
            "component_name",
            "previous_component",
            "position_x",
            "position_y",
        }

        component_data = model_to_dict(underlying_object, exclude=excluded_fields)
        param_strings = []

        for k, v in component_data.items():
            if v:
                if isinstance(v, str):
                    param_strings.append(
                        f"{k}=input_data{v}" if v.startswith(".") else f"{k}='{v}'",
                    )
                else:
                    param_strings.append(f"{k}={v}")

        if keyboard:
            param_strings.append("reply_markup=keyboard")
        if file_params:
            param_strings.append(file_params)

        return ", ".join(param_strings)

    def generate_code(self) -> str:
        if self.component_type != Component.ComponentType.TELEGRAM:
            raise NotImplementedError

        underlying_object = self.component_content_type.model_class().objects.get(
            pk=self.pk,
        )
        file_params = self._get_file_params(underlying_object)
        method = self._get_method_name(underlying_object.__class__.__name__)

        code = [
            f"async def {underlying_object.code_function_name}(input_data: Message, **kwargs):",
        ]

        # Handle code component
        if underlying_object.__class__.__name__ == "CodeComponent":
            code.extend(self._format_code_component(underlying_object))
            return "\\n".join(code)

        keyboard = None
        # Check if markup exists before accessing it
        if hasattr(underlying_object, "markup") and underlying_object.markup:
            markup = underlying_object.markup
            match markup.markup_type:
                case markup.MarkupType.ReplyKeyboard:
                    keyword_class = "ReplyKeyboardMarkup"
                    button_class = "KeyboardButton"
                case markup.MarkupType.InlineKeyboard:
                    keyword_class = "InlineKeyboardMarkup"
                    button_class = "InlineKeyboardButton"
                case _:
                    raise NotImplementedError(f"Unknown markup {markup.markup_type}")
            keyboard_buttons = "["
            for row in markup.buttons:
                keyboard_buttons += "[\\n"

                for cell in row:
                    args = {"text": cell}
                    if markup.markup_type == markup.MarkupType.InlineKeyboard:
                        args["callback_data"] = markup.get_callback_data(cell)

                    keyboard_buttons += f"{button_class}(\\n"
                    for k, v in args.items():
                        keyboard_buttons += f'{k} = "{v}"\\n'
                    keyboard_buttons += f")"

                keyboard_buttons += "]"
            keyboard_buttons += "]"
            keyboard = f"{keyword_class}(resize_keyboard=True, one_time_keyboard=False, keyboard = {keyboard_buttons})"

        # Generate parameters and method call
        params_str = self._get_component_params(
            underlying_object,
            keyboard,
            file_params,
        )
        code.append(f"    await bot.{method}({params_str})")

        # Handle next components
        for next_component in underlying_object.next_component.all():
            next_component = (
                next_component.component_content_type.model_class().objects.get(
                    pk=next_component.pk,
                )
            )
            code.append(
                f"    await {next_component.code_function_name}(input_data, **kwargs)",
            )

        return "\\n".join(code)

    def get_all_next_components(self) -> List["Component"]:
        ans = {}
        stack = [self]
        while stack:
            current = stack.pop()
            if current.id not in ans:
                ans[current.id] = current
                for next_component in current.next_component.all():
                    if next_component.id not in ans:
                        stack.append(next_component)
        return list(ans.values())


""")
    not_supported = []

    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        body = re.findall(table_body_pattern, method[2])
        file.write(f"class {name[0].upper()+name[1:]}(Component):\n")
        file.write(f"    \"\"\"{text_cleaner(comment)}\"\"\"\n\n")
        required_fields = []
        supported_reply_markup = False

        # print(f"class {name[0].upper()+name[1:]}(TelegramComponent):")
        for item in body:
            type_field = text_cleaner(item[1])
            django_field = ""
            keyboard_field = ""
            django_param = ""

            if item[2] == 'Optional' or item[0] == 'thumbnail':
                django_param = "null=True, blank=True,"
            elif item[2] == 'Yes':
                django_param = "null=True, blank=True,"
                required_fields.append(item[0])

            match = re.search(max_char_pattern, text_cleaner(item[3]))
            if match:
                extracted_number = int(match.group(1))  # Convert to integer
                django_param += f" max_length = {extracted_number},"

            django_param += f" help_text=\"{text_cleaner(item[3])}\""

            if type_field == 'String' or type_field == 'Integer or String':
                django_field = f" = models.CharField({django_param})"
            elif type_field == 'Boolean':
                django_field = f" = models.BooleanField({django_param})"
            elif type_field == 'Integer':
                django_field = f" = models.IntegerField({django_param})"
            elif type_field == 'Array of Integer':
                django_field = f"= ArrayField(models.IntegerField(), default=list, {django_param})"
            elif type_field == 'Array of String':
                django_field = f"= ArrayField(models.CharField(), default=list, {django_param})"
            elif type_field == 'Float':
                django_field = f" = models.FloatField({django_param})"
            elif type_field == 'InputFile or String':
                django_field = f" =  models.FileField(upload_to=\"{item[0]}/\", {django_param})"
            elif type_field == 'InlineKeyboardMarkup or ReplyKeyboardMarkup or ReplyKeyboardRemove or ForceReply':
                supported_reply_markup = True
            elif type_field == 'InlineKeyboardMarkup':
                supported_reply_markup = True
            else:
                if type_field not in not_supported:
                    not_supported.append(type_field)
                
            if django_field:
                file.write(f"    {item[0]}{django_field}\n")
            if keyboard_field:
                file.write(f"{keyboard_field}\n")

        file.write(f"    @property\n")
        file.write(f"    def required_fields(self) -> list:\n")
        file.write(f"        return {required_fields}\n")

        file.write(f"    @property\n")
        file.write(f"    def reply_markup_supported(self) -> bool:\n")
        file.write(f"        return {supported_reply_markup}\n")

        file.write("\n\n")
file.close()
print(f"not supported: {len(not_supported)}")
print(not_supported)

not supported: 26
['Array of MessageEntity', 'LinkPreviewOptions', 'ReplyParameters', 'Array of InputPaidMedia', 'Array of InputMediaAudio, InputMediaDocument, InputMediaPhoto and InputMediaVideo', 'Array of InputPollOption', 'Array of ReactionType', 'ChatPermissions', 'InputFile', 'Array of BotCommand', 'BotCommandScope', 'MenuButton', 'ChatAdministratorRights', 'InputMedia', 'InputProfilePhoto', 'AcceptedGiftTypes', 'InputStoryContent', 'Array of StoryArea', 'Array of InputSticker', 'InputSticker', 'MaskPosition', 'Array of InlineQueryResult', 'InlineQueryResultsButton', 'InlineQueryResult', 'Array of LabeledPrice', 'Array of ShippingOption']


In [47]:
with open('../component/telegram/serializers.py', 'w') as file:
    file.write("""from rest_framework import serializers
from component.telegram.models import *

class ModelSerializerCustom(serializers.ModelSerializer):
    def create(self, validated_data: dict) -> Component:
        validated_data["bot_id"] = self.context.get("bot")
        return super().create(validated_data)
""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        file.write(f"class {name[0].upper()+name[1:]}Serializer(ModelSerializerCustom):\n")
        file.write(f"    class Meta:\n")
        file.write(f"        model = {name[0].upper()+name[1:]}\n")
        # file.write(f"        depth = 1\n")
        file.write(f"        exclude = [\"bot\"]\n")
        file.write(f"        read_only_fields = [\"component_type\"]")
        file.write("\n\n")
file.close()

In [26]:
with open('../component/telegram/views.py', 'w') as file:
    file.write("""from django.db.models import QuerySet
from rest_framework.viewsets import ModelViewSet

from bot.permissions import IsBotOwner
from component.telegram.serializers import *
from iam.permissions import IsLoginedPermission


class ModelViewSetCustom(ModelViewSet):
    def get_serializer_context(self) -> dict:
        context = super().get_serializer_context()
        context["bot"] = self.kwargs.get("bot")
        return context

    def get_queryset(self) -> QuerySet:
        return super().get_queryset().filter(bot=self.kwargs.get("bot"))

""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        correct_name = name[0].upper()+name[1:]
        file.write(f"class {correct_name}ViewSet(ModelViewSetCustom):\n")
        file.write(f"    permission_classes = [IsLoginedPermission, IsBotOwner]\n")
        file.write(f"    serializer_class = {correct_name}Serializer\n")
        file.write(f"    queryset = {correct_name}.objects.all()\n")
        file.write("\n\n")
file.close()

In [28]:
with open('../component/telegram/urls.py', 'w') as file:
    file.write("""from django.urls import include, path
from rest_framework.routers import DefaultRouter

from component.telegram.views import *
router = DefaultRouter()

               
""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        correct_name = name[0].upper()+name[1:]
        under_lined_name = ""
        for char in name:
            if char.isupper():
                under_lined_name += "-"+char.lower()
            else:
                under_lined_name += char

        file.write(f"router.register(r\"{under_lined_name}\", {correct_name}ViewSet)\n")

    file.write("""

urlpatterns = [
    path("", include(router.urls)),
]
""")
file.close()

In [48]:
!python -m black ../component/*

/home/ali/.local/bin/Cursor-0.48.8-x86_64.AppImage: No module named black


In [50]:
!pre-commit run --all-files

isort....................................................................Passed
black....................................................................Passed
Add trailing commas......................................................Passed
trim trailing whitespace.................................................Passed
